# THE BIG BANG THEORY

<div style="display: flex; align-items: center;">
    <img src="https://upload.wikimedia.org/wikipedia/en/thumb/7/7b/The_Big_Bang_Theory_%28Official_Title_Card%29.png/330px-The_Big_Bang_Theory_%28Official_Title_Card%29.png" alt="The Big Bang Theory" style="width: 200px;"/>
    <p style="margin-left: 20px;"> "The Big Bang Theory" is a popular American television sitcom created by Chuck Lorre and Bill Prady. It originally aired on CBS from September 24, 2007, to May 16, 2019, spanning a total of twelve seasons. The show is known for its witty humor, clever writing, and endearing characters, particularly the group of socially awkward but brilliant physicists at Caltech University.
</p>
</div>




In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/The_Big_Bang_Theory'

# Fetch the HTML content of the Wikipedia page
response = requests.get(url)
if response.status_code == 200:
    html_content = response.text
else:
    print("Failed to retrieve page")

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table containing viewership statistics
table = soup.find('table', {'class': 'wikitable plainrowheaders'})

# Extract data from the table
data = []
for row in table.find_all('tr'):
    columns = row.find_all(['th', 'td'])
    if len(columns) == 8:  # Ensure it's a data row
        season = columns[0].text.strip()
        episodes = columns[1].text.strip()
        # Extract dates and format them as 'YYYY-MM-DD'
        first_aired = re.search(r'\d{4}-\d{2}-\d{2}', columns[2].text.strip()).group()
        last_aired = re.search(r'\d{4}-\d{2}-\d{2}', columns[3].text.strip()).group()
       
        viewers_rank = columns[4].text.strip()
        viewers_million = columns[5].text.strip()
        _18_49_rank = columns[6].text.strip()
        _18_49_rating_share = columns[7].text.strip()
        data.append((season, episodes, first_aired, last_aired, viewers_rank, viewers_million, _18_49_rank, _18_49_rating_share))

# Create DataFrame from the extracted data
df = pd.DataFrame(data, columns=['Season', 'Episodes', 'First aired', 'Last aired', 'Viewers rank', 'U.S. Viewers (millions)', '18–49 rank', '18–49 rating/share'])

# Print DataFrame
print(df)


   Season Episodes First aired  Last aired Viewers rank  \
0       1       17  2007-09-24  2008-05-19       68[20]   
1       2       23  2008-09-22  2009-05-11       40[22]   
2       3       23  2009-09-21  2010-05-24           12   
3       4       24  2010-09-23  2011-05-19           13   
4       5       24  2011-09-22  2012-05-10            8   
5       6       24  2012-09-27  2013-05-16            3   
6       7       24  2013-09-26  2014-05-15            2   
7       8       24  2014-09-22  2015-05-07            2   
8       9       24  2015-09-21  2016-05-12            2   
9      10       24  2016-09-19  2017-05-11            2   
10     11       24  2017-09-25  2018-05-10            1   
11     12       24  2018-09-24  2019-05-16            2   

   U.S. Viewers (millions) 18–49 rank 18–49 rating/share  
0                     8.34     46[21]              3.3/8  
1                    10.07          —                  —  
2                    14.22      5[23]             5.3/1